In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt 
from sklearn.preprocessing import OrdinalEncoder

import numpy as np 
import pandas as pd 


C:\Users\cataj\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sub = pd.read_csv('sample_submission.csv')

In [3]:
enc = OrdinalEncoder()
y = enc.fit_transform(df_train[['prognosis']])
y = tf.keras.utils.to_categorical(y)
y.shape

(707, 11)

In [4]:
X = df_train.drop(['prognosis'],axis=1) 

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=True, test_size=0.3)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


classes = ['Lyme_disease', 'Tungiasis', 'Zika', 'Rift_Valley_fever', 'West_Nile_fever', 'Malaria', 'Chikungunya', 'Plague', 'Dengue', 'Yellow_Fever', 'Japanese_encephalitis']

#  model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5)

# train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stop])

# predict probabilities
probs = model.predict(df_test)



Epoch 1/50
13/13 [==============================] - 2s 47ms/step - loss: 90.6799 - accuracy: 0.1038 - val_loss: 22.6961 - val_accuracy: 0.0808
Epoch 2/50
13/13 [==============================] - 0s 9ms/step - loss: 74.9978 - accuracy: 0.0962 - val_loss: 9.9130 - val_accuracy: 0.0909
Epoch 3/50
13/13 [==============================] - 0s 9ms/step - loss: 59.1638 - accuracy: 0.0734 - val_loss: 7.0851 - val_accuracy: 0.0707
Epoch 4/50
13/13 [==============================] - 0s 11ms/step - loss: 47.9520 - accuracy: 0.0962 - val_loss: 4.9343 - val_accuracy: 0.0606
Epoch 5/50
13/13 [==============================] - 0s 9ms/step - loss: 35.5126 - accuracy: 0.1089 - val_loss: 6.3407 - val_accuracy: 0.0707
Epoch 6/50
13/13 [==============================] - 0s 10ms/step - loss: 29.6062 - accuracy: 0.0937 - val_loss: 5.2704 - val_accuracy: 0.0808
Epoch 7/50
13/13 [==============================] - 0s 9ms/step - loss: 23.0287 - accuracy: 0.0810 - val_loss: 4.3397 - val_accuracy: 0.0808
Epoch 8/5

In [7]:
# top 3 prediction for each sample

top3 = []
for prob in probs:
    indices = prob.argsort()[-3:][::-1]
    top3.append(indices)

# convert indices to class labels
class_labels = np.array(classes)
top3_pred_class = class_labels[top3]



In [8]:
# save prediction
with open('predictions.csv', 'w') as f:
    f.write('Patient_Id,Prognosis1,Prognosis2,Prognosis3\n')
    for i, pred in enumerate(top3_pred_class):
        f.write(f'{i+1},{pred[0]},{pred[1]},{pred[2]}\n')


In [9]:
df_pred = pd.read_csv('predictions.csv')
df_pred.head(5)

,Patient_Id,Prognosis1,Prognosis2,Prognosis3
0,1,Dengue,Zika,Plague
1,2,Dengue,Zika,Plague
2,3,Dengue,Zika,Plague
3,4,Dengue,Zika,Plague
4,5,Dengue,Zika,Plague


In [10]:
df_pred['prognosis'] = df_pred.apply(lambda x: x['Prognosis1'] + ', ' + x['Prognosis2'] + ', ' + x['Prognosis3'], axis=1)

df_pred = df_pred.drop(['Prognosis1', 'Prognosis2', 'Prognosis3'], axis=1)


In [11]:
df_sub['prognosis'] = df_pred['prognosis']
df_sub.to_csv('submission.csv',index=False)
df_sub.head(5)

,id,prognosis
0,707,"Dengue, Zika, Plague"
1,708,"Dengue, Zika, Plague"
2,709,"Dengue, Zika, Plague"
3,710,"Dengue, Zika, Plague"
4,711,"Dengue, Zika, Plague"
